In [ ]:
!pip install mediapipe opencv-python pandas scikit-learn jdata numpy

In [1]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import jdata as jd
from mediapipe.framework.formats import landmark_pb2
import numpy as np

In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
atlas10_5=jd.load('atlas_19-5_10-5.json')

In [3]:
def affinemap(pfrom, pto):
    bsubmat=np.eye(3)
    ptnum=len(pfrom)
    amat=np.zeros((ptnum*3,9))
    for i in range(ptnum):
        amat[((i+1)*3-3):((i+1)*3),:]=np.kron(bsubmat,pfrom[i,:])
    amat=np.hstack((amat,np.tile(bsubmat,(ptnum,1))))
    
    bvec=np.reshape(pto,(ptnum*3,1))
    x=np.linalg.lstsq(amat, bvec, rcond=None)[0]
    A=np.reshape(x[0:9],(3,3))
    b=x[9:12]
    return [A, b]

def reg1020(Amat, bvec, pts):
    newpt = np.matmul(Amat, (np.array(pts)).T)
    newpt = newpt + np.tile(bvec, (1, len(pts)))
    newpt = newpt.T
    return newpt

def landmark2numpy(landmarks):
    pts = []
    for p in landmarks.landmark:
        pts.append([p.x, p.y, p.z])
    return np.array(pts)

def numpy2landmark(pts):
    landmarks = landmark_pb2.NormalizedLandmarkList(landmark=[])
    for p in pts:
         lm =  landmark_pb2.NormalizedLandmark(x=p[0], y=p[1], z=p[2])
         landmarks.landmark.append(lm)
    return landmarks

In [21]:
cap = cv2.VideoCapture(-1)
#cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
#cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5, smooth_landmarks=True) as holistic:

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False    

        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if(results.face_landmarks is not None):
            pts=results.face_landmarks.landmark
            landmark_face_subset = landmark_pb2.NormalizedLandmarkList(
                landmark=[
                    pts[168],
                    pts[10],
                    {'x':2*pts[234].x-pts[227].x, 'y':2*pts[234].y-pts[227].y, 'z':2*pts[234].z-pts[227].z},
                    {'x':2*pts[454].x-pts[447].x, 'y':2*pts[454].y-pts[447].y, 'z':2*pts[454].z-pts[447].z}
                ])
            #print(landmark2numpy(landmark_face_subset))
            #print(np.array([atlas10_5['nz'], atlas10_5['sm'][0], atlas10_5['rpa'], atlas10_5['lpa']]))
            Amat, bvec = affinemap(np.array([atlas10_5['nz'], atlas10_5['sm'][1], atlas10_5['rpa'], atlas10_5['lpa']]),
                                   landmark2numpy(landmark_face_subset))

            aal = reg1020(Amat, bvec, atlas10_5['aal'])
            aar = reg1020(Amat, bvec, atlas10_5['aar'])
            sm = reg1020(Amat, bvec, atlas10_5['sm'])
            cm = reg1020(Amat, bvec, atlas10_5['cm'])

            
        # 1. Draw face landmarks
        facemesh = mp_holistic.FACE_CONNECTIONS if 'FACE_CONNECTIONS' in dir(mp_holistic) else mp_holistic.FACEMESH_CONTOURS
        mp_drawing.draw_landmarks(image, results.face_landmarks, facemesh, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )   
        if(results.face_landmarks is not None):
            mp_drawing.draw_landmarks(image, landmark_face_subset, None, 
                                 mp_drawing.DrawingSpec(color=(255,0,0), thickness=1, circle_radius=3),
                                 mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=3)
                                 )   
            mp_drawing.draw_landmarks(image, numpy2landmark(np.vstack((aal,aar,sm,cm))), None, 
                                 mp_drawing.DrawingSpec(color=(255,255,0), thickness=1, circle_radius=3),
                                 mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=3)
                                 )   
        cv2.imshow('Raw Webcam Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()